In [5]:
%load_ext autoreload
%autoreload 2
import parse_agent,pde_descriptions,system_prompt
import os
from dotenv import load_dotenv
from google import genai
from google.genai.types import GenerateContentConfig
import markdown_to_json
import json

load_dotenv()
api_key = os.getenv("GEMINI_API_KEY")


fk_description = pde_descriptions.fk_description
tau_d = 0.5714
pde_description = fk_description.format(tau_d=tau_d)

parse_prompt = parse_agent.parse_prompt
system_prompt = system_prompt.system_prompt

user_prompt = parse_prompt.format(
    user_input=pde_description,
)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
client = genai.Client(api_key=api_key)

response = client.models.generate_content(
    model="gemini-2.5-pro", 
    config=GenerateContentConfig(
            system_instruction=system_prompt
        ),
    contents=user_prompt    
    )

print(response.text)

ClientError: 400 INVALID_ARGUMENT. {'error': {'code': 400, 'message': 'API Key not found. Please pass a valid API key.', 'status': 'INVALID_ARGUMENT', 'details': [{'@type': 'type.googleapis.com/google.rpc.ErrorInfo', 'reason': 'API_KEY_INVALID', 'domain': 'googleapis.com', 'metadata': {'service': 'generativelanguage.googleapis.com'}}, {'@type': 'type.googleapis.com/google.rpc.LocalizedMessage', 'locale': 'en-US', 'message': 'API Key not found. Please pass a valid API key.'}]}}

In [ ]:
# Simple one-liner to save the response

#deleat ```json at beginning of response.text if exist
response_text = response.text
if response_text.startswith("```json"):
    response_text = response_text[len("```json"):]

#deleat ``` at ending of response.text if exist
if response_text.endswith("```"):
    response_text = response_text[:-3]

with open("resp.json", "w", encoding="utf-8") as f:
    f.write(response_text)